# 재고수불부 분석

In [17]:
import pandas as pd

# 파일을 UTF-8 인코딩으로 불러옵니다.
df_current = pd.read_csv('재고수불부_당기.csv', encoding='utf-8')
df_previous = pd.read_csv('재고수불부_전기.csv', encoding='utf-8')


print("### 당기 재고수불부 데이터 정보 (UTF-8 인코딩) ###")
print(df_current.head())
print(df_current.info())


print("\n### 전기 재고수불부 데이터 정보 (UTF-8 인코딩) ###")
print(df_previous.head())
print(df_previous.info())

### 당기 재고수불부 데이터 정보 (UTF-8 인코딩) ###
     기준년월 기간귀속   플랜트 자재유형         자재코드     계정코드  단위  기초수량          기초단가  \
0  201612   당기  1000  부재료  77113-2L000  1120500  EA   613   1357.851550   
1  201612   당기  1000  부재료  69300-3L300  1120500  EA     0      0.000000   
2  201612   당기  1000  부재료  71660-1C500  1120500  EA     0  19762.000000   
3  201612   당기  1000  부재료  64110-2C010  1120500  EA     0      0.000000   
4  201612   당기  1000  부재료  71630-3D000  1120500  EA     7   5547.571429   

     기초금액  ...  이동출고수량  이동출고단가  이동출고금액  기타출고수량    기타출고단가  기타출고금액  기말수량  \
0  832363  ...       0       0       0       0   1230.85       0    43   
1       0  ...       0       0       0       0  18392.17       0   -27   
2       0  ...       0       0       0       0  19762.00       0   -35   
3       0  ...       0       0       0       0   5199.90       0    12   
4   38833  ...       0       0       0       0   6381.65       0    11   

       기말단가    기말금액  비고  
0   1230.86   52927   0  
1  18392.00 -496

## 1.전기말, 당기초 재고수량 대사

In [27]:
# 필요한 컬럼만 선택하여 데이터프레임을 준비합니다.
# 당기(current) 데이터의 '기초수량'과 전기(previous) 데이터의 '기말수량'을 비교해야 하므로,
# 당기 데이터의 '기초수량' 컬럼 이름을 '기초수량_당기'로 변경하여 혼동을 막습니다.
df_current['기초수량_당기'] = df_current['기초수량']
df_previous['기말수량_전기']= df_previous['기말수량']
df_previous_selected = df_previous[['플랜트', '자재코드', '기말수량_전기']]
df_current_selected = df_current[['플랜트', '자재코드', '기초수량_당기']]

# 두 데이터프레임을 '플랜트'와 '자재코드'를 키로 inner join 합니다.

df_merged = pd.merge(df_previous_selected, df_current_selected, on=['플랜트', '자재코드'], how='inner')

# 전기 기말수량과 당기 기초수량이 일치하지 않는 자재코드를 필터링합니다.
df_discrepancy = df_merged[df_merged['기말수량_전기'] != df_merged['기초수량_당기']]
len_df_discrepancy = len(df_discrepancy)

# 결과 출력
if not df_discrepancy.empty:
    print("기말 재고수량과 기초 재고수량이 일치하지 않는 자재코드 목록:")
    print(df_discrepancy[['자재코드', '기말수량_전기', '기초수량_당기']])
    print("불일치 자재코드 수:" , len_df_discrepancy, "개")
else:
    print("전기 기말 재고수량과 당기 기초 재고수량이 일치하지 않는 자재코드는 없습니다.")


기말 재고수량과 기초 재고수량이 일치하지 않는 자재코드 목록:
           자재코드  기말수량_전기  기초수량_당기
2   71660-1C500       38        0
17  69300-3K002       66        0
불일치 자재코드 수: 2 개


## 2.입고,출고 수불부대사

In [43]:
print("### 1. 재고자산 수불부 대사 ###")

# 전기 재고수불부 대사
#(기초수량 + 총 입고수량) - (기말수량 + 총 출고수량) = 0
inflow_previous = df_previous['생산입고수량'] + df_previous['구매입고수량'] + df_previous['이동입고수량'] + df_previous['기타입고수량']
outflow_previous = df_previous['생산출고수량'] + df_previous['판매출고수량'] + df_previous['이동출고수량'] + df_previous['기타출고수량']
reconciliation_previous = df_previous['기초수량'] + inflow_previous - (df_previous['기말수량'] + outflow_previous)
reconciliation_previous_list = df_previous[reconciliation_previous.round(5) != 0]
# 소수점 반올림을 통해 미세한 부동 소수점 오차를 제거
num_previous_discrepancies = (reconciliation_previous.round(5) != 0).sum()
print(f"전기(2015년) 재고수불부에서 수불이 일치하지 않는 항목 수: {num_previous_discrepancies}개")
if reconciliation_previous_list.empty:
    print("전기 재고수불부에서는 일치하지 않는 항목이 없습니다.")
else:
    print("전기 재고수불부 불일치 목록: \n",reconciliation_previous_list)

# 당기 재고수불부 대사
inflow_current = df_current['생산입고수량'] + df_current['구매입고수량'] + df_current['이동입고수량'] + df_current['기타입고수량']
outflow_current = df_current['생산출고수량'] + df_current['판매출고수량'] + df_current['이동출고수량'] + df_current['기타출고수량']
reconciliation_current = df_current['기초수량'] + inflow_current - (df_current['기말수량'] + outflow_current)
reconciliation_current_list = df_current[reconciliation_current.round(5) != 0]
num_current_discrepancies = (reconciliation_current.round(5) != 0).sum()
print(f"\n당기(2016년) 재고수불부에서 수불이 일치하지 않는 항목 수: {num_current_discrepancies}개")
if reconciliation_current_list.empty:
    print("당기 재고수불부에서는 일치하지 않는 항목이 없습니다.")
else:
    print("당기 재고수불부 불일치 목록: \n", reconciliation_current_list)


### 1. 재고자산 수불부 대사 ###
전기(2015년) 재고수불부에서 수불이 일치하지 않는 항목 수: 0개
전기 재고수불부에서는 일치하지 않는 항목이 없습니다.

당기(2016년) 재고수불부에서 수불이 일치하지 않는 항목 수: 1개
당기 재고수불부 불일치 목록: 
        기준년월 기간귀속   플랜트 자재유형         자재코드     계정코드  단위  기초수량  기초단가  기초금액  ...  \
145  201612   당기  4000   제품  64300-3X952  1120100  EA     0   0.0     0  ...   

     이동출고단가  이동출고금액  기타출고수량    기타출고단가  기타출고금액  기말수량      기말단가     기말금액  비고  \
145       0       0       0  36371.05       0   209  36371.05  7601550   0   

     기초수량_당기  
145        0  

[1 rows x 39 columns]


## 3.재고자산 회전율 분석

In [51]:
print("\n### 재고자산 회전율 분석 ###")

print("### 1. 재고자산 회전율 분석 (전체) ###")
# 매출원가(Cost of Goods Sold) 계산: 기초재고금액 + 총 입고금액 - 기말재고금액
cogs_previous = df_previous['기초금액'] + (df_previous['생산입고금액'] + df_previous['구매입고금액'] + df_previous['이동입고금액'] + df_previous['기타입고금액']) - df_previous['기말금액']
cogs_current = df_current['기초금액'] + (df_current['생산입고금액'] + df_current['구매입고금액'] + df_current['이동입고금액'] + df_current['기타입고금액']) - df_current['기말금액']

# 평균 재고자산(Average Inventory) 계산: (기초재고금액 + 기말재고금액) / 2
avg_inv_previous = (df_previous['기초금액'] + df_previous['기말금액']) / 2
avg_inv_current = (df_current['기초금액'] + df_current['기말금액']) / 2

# 재고자산 회전율 계산
# 0으로 나누는 오류를 방지하기 위해 0인 경우 0으로 처리합니다, 결측값인 경우 0으로 표시합니다.
inventory_turnover_previous = (cogs_previous / avg_inv_previous).fillna(0).replace([float('inf'), -float('inf')], 0)
inventory_turnover_current = (cogs_current / avg_inv_current).fillna(0).replace([float('inf'), -float('inf')], 0)

# 전체 기간의 평균 회전율 계산
total_inventory_turnover_previous = inventory_turnover_previous.mean()
total_inventory_turnover_current = inventory_turnover_current.mean()

print(f"전기(2015년) 재고자산 회전율: {total_inventory_turnover_previous:.2f}")
print(f"당기(2016년) 재고자산 회전율: {total_inventory_turnover_current:.2f}")


print("\n### 2. 재고자산 회전율 분석 (특정 자재) ###")

#자재코드 리스트
unique_material_codes = df_current['자재코드'].unique()
print("입력가능한 자재코드 :", unique_material_codes)

def calculate_inventory_turnover(df, item_code=None):
    """
    주어진 데이터프레임에서 재고자산 회전율을 계산하는 함수.
    특정 자재코드를 지정하면 해당 자재에 대한 회전율을 계산합니다.
    """
    if item_code:
        df_filtered = df[df['자재코드'] == item_code].copy()
        if df_filtered.empty:
            return None
    else:
        df_filtered = df.copy()

    # 총 입고금액 계산 axis=1 활용하여 수평합
    total_inflow_amount = df_filtered[['생산입고금액', '구매입고금액', '이동입고금액', '기타입고금액']].sum(axis=1)

    # 매출원가(Cost of Goods Sold) 계산: 기초재고금액 + 총 입고금액 - 기말재고금액
    cogs = df_filtered['기초금액'] + total_inflow_amount - df_filtered['기말금액']

    # 평균 재고자산(Average Inventory) 계산: (기초재고금액 + 기말재고금액) / 2
    avg_inv = (df_filtered['기초금액'] + df_filtered['기말금액']) / 2

    # 재고자산 회전율 계산
    # 0으로 나누는 오류를 방지하기 위해 0인 경우 0으로 처리합니다.
    inventory_turnover = (cogs / avg_inv).fillna(0).replace([float('inf'), -float('inf')], 0)

    # 전체 기간의 평균 회전율 반환
    return inventory_turnover.mean()

# 특정 자재코드를 예시로 들어 분석
sample_item_code = '71201-2W000'
item_turnover_previous = calculate_inventory_turnover(df_previous, sample_item_code)
item_turnover_current = calculate_inventory_turnover(df_current, sample_item_code)

if item_turnover_previous is not None:
    print(f"전기(2015년) 자재코드 '{sample_item_code}'의 재고자산 회전율: {item_turnover_previous:.2f}")
else:
    print(f"전기(2015년) 자재코드 '{sample_item_code}'에 대한 데이터가 없습니다.")

if item_turnover_current is not None:
    print(f"당기(2016년) 자재코드 '{sample_item_code}'의 재고자산 회전율: {item_turnover_current:.2f}")
else:
    print(f"당기(2016년) 자재코드 '{sample_item_code}'에 대한 데이터가 없습니다.")



### 재고자산 회전율 분석 ###
### 1. 재고자산 회전율 분석 (전체) ###
전기(2015년) 재고자산 회전율: 123.10
당기(2016년) 재고자산 회전율: 120.17

### 2. 재고자산 회전율 분석 (특정 자재) ###
입력가능한 자재코드 : ['77113-2L000' '69300-3L300' '71660-1C500' '64110-2C010' '71630-3D000'
 '71202-2D010' '64140-2C700' '71630-3L000' '64140-2C010' '69141-3N700'
 '64140-2C000' '71660-26000' '71650-26000' '64300-4W000' '64120-26201'
 '64200-B1900' '69300-2H000' '69300-3K002' '71660-1C000' '69300-3D000'
 '65180-2B210' '69100-2L010' '64100-27000' '71754-2E000' '69131-3N000'
 '69141-3N000' '64500-4A010' '71603-1R401' '71633-2W000' '71643-2W000'
 '69300-2Q001' '71641-2W250Y' '71641-2W000Y' '71641-2W050Y' '71650-1C500'
 '71575-2H100' '64502-1E000' '69300-2Q000' '77122-2L200' '71402-2L000'
 '71202-2S010' '76123-39000' '69300-2H010' '71201-2Q000' '64300-3X902'
 '71202-2Q000' '69300-3L250' '71201-2S010' '71402-3N800' '71401-3N800'
 '64160-2C001' '71640-2V900' '69100-2H020' '71575-2H600' '77123-2L000'
 '71401-2L000' '71661-2W000Y' '71651-2W000Y' '71630-2V900' '71201-2S

## 추가. 자재코드 입력받아 실행하기

In [57]:
# 파일명과 자재코드를 입력받아 분석하는 새로운 함수를 정의합니다.
def analyze_turnover_by_file(filename, material_code=None):
    """
    지정된 파일을 불러와 재고자산 회전율을 분석하는 함수.
    """
    try:
        df = pd.read_csv(filename, encoding='utf-8')
        
        # calculate_turnover 함수를 호출하여 회전율 계산
        turnover_rate = calculate_inventory_turnover(df, material_code)
        
        if material_code:
            print(f"파일 '{filename}'의 자재코드 '{material_code}'에 대한 재고자산 회전율: {turnover_rate:.2f}")
        else:
            print(f"파일 '{filename}'의 전체 재고자산 회전율: {turnover_rate:.2f}")
            
    except FileNotFoundError:
        print(f"오류: '{filename}' 파일을 찾을 수 없습니다.")
    except KeyError as e:
        print(f"오류: 파일에 필요한 컬럼이 없습니다. ({e})")
        
# 함수 사용 예시
print("### 새로운 함수를 사용한 재고자산 회전율 분석 ###")
analyze_turnover_by_file('재고수불부_당기.csv')
analyze_turnover_by_file('재고수불부_전기.csv', material_code='64300-3X952')
analyze_turnover_by_file('재고수불부_당기.csv', material_code='77113-2L000')


### 새로운 함수를 사용한 재고자산 회전율 분석 ###
파일 '재고수불부_당기.csv'의 전체 재고자산 회전율: 120.17
파일 '재고수불부_전기.csv'의 자재코드 '64300-3X952'에 대한 재고자산 회전율: 22.66
파일 '재고수불부_당기.csv'의 자재코드 '77113-2L000'에 대한 재고자산 회전율: 7.42


In [64]:
# =========================================================
# 사용자 입력 받기
# =========================================================
import sys

# 1. 파일명 입력 받기
file_name = input("분석할 CSV 파일명을 입력하세요 (예: 재고수불부_당기.csv): ")
try:
    # 2. 입력받은 파일명으로 파일을 읽어 DataFrame으로 만듭니다.
    #    이제 'df' 변수는 DataFrame이 됩니다.
    df = pd.read_csv(file_name, encoding='utf-8')

    # 3. DataFrame(df)에서 '자재코드' 컬럼의 고유값을 가져옵니다.
    unique_material_codes_2 = df['자재코드'].unique()

    print("입력 가능한 자재코드 목록 (총 {}개):".format(len(unique_material_codes_2)))
    print(unique_material_codes)

except FileNotFoundError:
    print(f"오류: '{file_name}' 파일을 찾을 수 없습니다. 파일명을 다시 확인해주세요.")
    sys.exit() # 파일을 찾을 수 없으면 여기서 프로그램 종료
except KeyError:
    print("오류: 파일에 '자재코드' 컬럼이 존재하지 않습니다.")
    sys.exit() # '자재코드' 컬럼이 없으면 프로그램 종료
except Exception as e:
    print(f"오류가 발생했습니다: {e}")
    sys.exit() # 그 외 예외가 발생하면 프로그램 종료
# 2. 자재코드 입력 받기 (선택 사항)
material_code = input("분석할 자재코드를 입력하세요 (예: 71660-2W000) (전체 분석 시 Enter): ")

if not material_code:
    material_code = None

# =========================================================
# 분석 실행 및 결과 출력
# =========================================================
if material_code:
    result = analyze_turnover_by_file(file_name, material_code=material_code)
    if result is not None:
        print(f"'{file_name}' 파일에서 '{material_code}'의 재고자산 회전율: {result:.2f}")
else:
    result = analyze_turnover_by_file(file_name)
    if result is not None:
        print(f"'{file_name}' 파일의 전체 재고자산 회전율: {result:.2f}")

분석할 CSV 파일명을 입력하세요 (예: 재고수불부_당기.csv):  d


오류: 'd' 파일을 찾을 수 없습니다. 파일명을 다시 확인해주세요.


SystemExit: 

C:\Users\user\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## 추가2. 입력 받을때까지 종료 X

In [70]:
# 1. 파일명 입력 받기 (반복문으로 감싸서 실패 시 재입력 요청)
while True:
    file_name = input("분석할 CSV 파일명을 입력하세요 (예: 재고수불부_당기.csv): ")

    try:
        # 입력받은 파일명으로 파일을 읽어 DataFrame으로 만듭니다.
        df = pd.read_csv(file_name, encoding='utf-8')
        
        # 파일 읽기 성공 시 반복문 탈출
        break

    except FileNotFoundError:
        print(f"오류: '{file_name}' 파일을 찾을 수 없습니다. 파일명을 다시 확인해주세요.")
    except KeyError:
        print("오류: 파일에 '자재코드' 컬럼이 존재하지 않습니다.")
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")

# 2. DataFrame(df)에서 '자재코드' 컬럼의 고유값을 가져옵니다.
try:
    unique_material_codes_3 = df['자재코드'].unique()
    print("입력 가능한 자재코드 목록 (총 {}개):".format(len(unique_material_codes_3)))
    print(unique_material_codes_3)
except KeyError:
    print("오류: 파일에 '자재코드' 컬럼이 존재하지 않습니다. 프로그램을 종료합니다.")
    sys.exit()

# 3. 자재코드 입력 받기 (선택 사항)
while True:
    material_code = input("분석할 자재코드를 입력하세요 (예:71630-2V000 / 전체 분석 시 Enter): ")

    if not material_code:
        # 전체 분석을 선택한 경우
        material_code = None
        break
    elif material_code in unique_material_codes:
        # 유효한 자재코드를 입력한 경우
        break
    else:
        # 유효하지 않은 자재코드를 입력한 경우
        print(f"오류: 입력한 '{material_code}'는 유효하지 않은 자재코드입니다. 목록에서 확인 후 다시 입력해주세요.")

    
# =========================================================
# 분석 실행 및 결과 출력
# =========================================================
if material_code:
    result = analyze_turnover_by_file(file_name, material_code=material_code)
    if result is not None:
        print(f"'{file_name}' 파일에서 '{material_code}'의 재고자산 회전율: {result:.2f}")
else:
    result = analyze_turnover_by_file(file_name)
    if result is not None:
        print(f"'{file_name}' 파일의 전체 재고자산 회전율: {result:.2f}")

분석할 CSV 파일명을 입력하세요 (예: 재고수불부_당기.csv):  재고


오류: '재고' 파일을 찾을 수 없습니다. 파일명을 다시 확인해주세요.


분석할 CSV 파일명을 입력하세요 (예: 재고수불부_당기.csv):  재고수불부_당기


오류: '재고수불부_당기' 파일을 찾을 수 없습니다. 파일명을 다시 확인해주세요.


분석할 CSV 파일명을 입력하세요 (예: 재고수불부_당기.csv):  재고수불부_당기.csv


입력 가능한 자재코드 목록 (총 175개):
['77113-2L000' '69300-3L300' '71660-1C500' '64110-2C010' '71630-3D000'
 '71202-2D010' '64140-2C700' '71630-3L000' '64140-2C010' '69141-3N700'
 '64140-2C000' '71660-26000' '71650-26000' '64300-4W000' '64120-26201'
 '64200-B1900' '69300-2H000' '69300-3K002' '71660-1C000' '69300-3D000'
 '65180-2B210' '69100-2L010' '64100-27000' '71754-2E000' '69131-3N000'
 '69141-3N000' '64500-4A010' '71603-1R401' '71633-2W000' '71643-2W000'
 '69300-2Q001' '71641-2W250Y' '71641-2W000Y' '71641-2W050Y' '71650-1C500'
 '71575-2H100' '64502-1E000' '69300-2Q000' '77122-2L200' '71402-2L000'
 '71202-2S010' '76123-39000' '69300-2H010' '71201-2Q000' '64300-3X902'
 '71202-2Q000' '69300-3L250' '71201-2S010' '71402-3N800' '71401-3N800'
 '64160-2C001' '71640-2V900' '69100-2H020' '71575-2H600' '77123-2L000'
 '71401-2L000' '71661-2W000Y' '71651-2W000Y' '71630-2V900' '71201-2S300'
 '64120-3A001' '64300-2S800' '71201-2S310' '71202-2S310' '71606-3J200'
 '76214-39000' '76224-39000' '71620-B8200' '716

분석할 자재코드를 입력하세요 (예:71630-2V000 / 전체 분석 시 Enter):  71601-3Z300


파일 '재고수불부_당기.csv'의 자재코드 '71601-3Z300'에 대한 재고자산 회전율: 116.14
